# Quickstart

## Installation

CredoAI Lens can be installed with pip from PyPI as follows:

`pip install credoai`

Optional dependencies can be installed as follows:

`pip install credoai[extras]`

## Overview

CredoAI Lens is the assessment framework component of the broader CredoAI suite.
It is usable as a standalone gateway to assessments or in combination
with CredoAI's Governance Platform. 

Understanding how your AI systems are operating is the most important step in intervening upon your system. From the technically complicated questions of improving a system to the business-relevant questions of whether to deploy a system - everything is fundamentally founded upon good observability. Lens strives to make assessment comprehensive, easy, and adaptable.

### Assessments

CredoAI Lens is an entry point to assessments developed by CredoAI, as well as the broader ecosystem of open-source assessments. Custom analytics can also be folded in easily (see the `lens customization` notebook)

AI system assessment starts with verifying standard performance metrics to an evolving set of assessments falling under the banner of *Responsible AI*. A non-exhaustive list includes

* Fairness
* Explainability
* Performance
* Robustness

These different categories of assessment differ substantially based on whether one is 
evaluating datasets or models, what kind of model (e.g., tabular, NLP, computer vision), and the use-case. As the ecosystem develops, Lens will support assessing a broader range of AI systems. Currently, we are focused on Fairness.

### Governance

While Lens is a stand-alone assessment framework, its value is increased when combined with the CredoAI Governance Platform. The platform supports multi-stakeholder `Alignment` on how to assess your AI systems (e.g., what does good look like for this system?). It also supports translating assessment results into a Risk perspective that is scalable across your organization and understandable to diverse stakeholders.



## Lens in 5 minutes

Get your data and models together! Here we have a support vector machine trained on the Iris Dataset. No train/test split needed for this tutorial!

In [1]:
# imports for example data and model training
from credoai.data import fetch_creditdefault
from sklearn.ensemble import GradientBoostingClassifier

# Base Lens imports
import credoai.lens as cl

In [2]:
data = fetch_creditdefault(as_frame=True)
X = data['data'].drop(columns=['SEX'])
y = data['target']
sensitive_feature = data['data']['SEX']

In [3]:
model = GradientBoostingClassifier()
model.fit(X,y)

GradientBoostingClassifier()

### Using Lens

Below is a basic example where our goal is to evaluate the fairness of the above model. We will rely on Lens defaults for this analysis.

#### Preparing artifacts

Lens interacts with Credo Artifacts which wrap models and datasets and standardizes them for use by different assessments.
Below we create a `CredoModel` object, which automatically infers that the "model" object is from scikit-learn. We also create a `CredoData` object which is store X, y and sensitive features. Both of these objects are customizable. See `lens_customization.ipynb` for more information.

In [4]:
credo_model = cl.CredoModel(name='credit_default_classifier',
                            model=model)

credo_data = cl.CredoData(name='UCI-credit-default',
                          X=X, 
                          y=y.astype(int),
                          sensitive_features=sensitive_feature)

The functionality of these artifacts determines which assessments can be run. Lens can automatically determine which assessments are runnable. In this case the Dataset Assessment and FairnessBase Assessment can be run. You can see what assessments are runnable with the following function.

In [10]:
from credoai.assessment import get_usable_assessments
get_usable_assessments(credo_model, credo_data)

{'Dataset': <credoai.assessment.assessments.DatasetAssessment at 0x16666e8b0>,
 'FairnessBase': <credoai.assessment.assessments.FairnessBaseAssessment at 0x16666ebe0>}

You can also list all assesments, along with their names.

In [13]:
from credoai.assessment.utils import get_assessment_names
get_assessment_names()

{'DatasetAssessment': 'Dataset',
 'FairnessBaseAssessment': 'FairnessBase',
 'NLPEmbeddingBiasAssessment': 'NLPEmbeddingBias',
 'NLPGeneratorAssessment': 'NLPGenerator'}

#### Alignment Spec

The Alignment Spec describes how the assessments should be run at a high level. It should be thought of as the output of a multi-stakeholder alignment session on "how we should assess this AI system". Another way to think about it is as the *parameterization* of the assessments Lens will run.

If you use the Credo AI Governance Platform, the alignment spec is a principle artifact determined during the *Alignment Phase*. It is the output of multi-stakeholder collaboration. Lens will automatically download the Alignment Spec associated with your governance credentials (which uses another artifact: `CredoGovernance`)

The Alignment Spec retrieved from the Governance Platform is a *subset* of all possible parameterizations of the different assessments you can run. Some parameters are too "in the weeds" for multistakeholder collaboration! However, you have full control of the assessments by manually defining the `spec`. Anything defined in the spec parameter will take precedence over the spec retrieved from the Governance Platform.

**Setting up the Spec**

The alignment spec is a set of {assessment_name: parameter} pairs. The assessment name must be the name of one of the assessments, as returned by `get_usable_assessments` (above). In general, the name will be the name of the method without the trailing "assessment". For example, FairnessBaseAssessment -> "FairnessBase". `get_assessment_names` will tell you the names you need.

Not all assessments *require* a spec, though many can be customized. In the case of "FairnessBase" a spec defining a list of metrics **is** required.

In [7]:
# specify the metrics that will be used by the FairnessBase assessment
alignment_spec = {
    'FairnessBase': {'metrics': ['precision_score']}
}

#### Run Lens

Once we have the model and data artifacts, as well as the spec, we can run Lens. By default it will automatically infer which assessments to run, just as we manually did above!

In [8]:
lens = cl.Lens(model=credo_model,
               data=credo_data,
               spec=alignment_spec)

Lens has one primary method: `run_assessments`

`run_assessments` outputs the results into a dictionary that can be used for further processing. If the `export` keyword is used, it can also export the data to a json or straight to Credo AI's governance platform (again, if you are using it!)

In [9]:
results = lens.run_assessments()
results.keys()

dict_keys(['Dataset', 'FairnessBase'])

In [ ]:
results['FairnessBase']['fairness']

Example export to `~/credoai_test_location`. Uncomment to test

In [ ]:
# # example export
# import os
# output_directory = os.path.join(os.path.expanduser('~'), 'credoai_test_location')
# lens.run_assessments(export=output_directory)